In [99]:
import textract
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import RegexpTokenizer
from collections import Counter

In [123]:
epub_path = "/Users/katbishop/Desktop/DSI-SF2-bishopkd/projects/capstone/data/_epub_working/"
txt_path = "/Users/katbishop/Desktop/DSI-SF2-bishopkd/projects/capstone/data/_txt/"

In [124]:
# function to extract text from epub

def convert_epub_to_text(epub_path, epub_file, txt_path):

    clean_text = ''
    text_name = epub_file.replace(' ','_')[:-4]+'txt'
    
    # extract text from epub
    text = textract.process(epub_path+epub_file,encoding='utf_8')
    
    # trip out the unicode and return characters
    # still working on the \ characters
    #for i in text.split(' '):
    clean_text = text.decode('ascii', 'ignore').replace('\n',' ')
    
    # save as text file
    text_file = open(txt_path+text_name, 'w')
    text_file.write(clean_text)
    text_file.close()
    

In [80]:
# loop through files in directory, convert file, save file in new folder

for epub in os.listdir(epub_path):
    try:
        convert_epub_to_text(epub_path, epub, txt_path)
    except:
        print epub, " failed"

Sacred Games - Vikram Chandra.epub  failed


In [125]:
# load txt files into dataframe, 
# give each entry a best_selling 1/0 entry and a sci_fi 1/0 (0=romance) indicator

path = '/Users/katbishop/Desktop/DSI-SF2-bishopkd/projects/capstone/data/'
folders = ['sci-fi_top','sci-fi_flop','romance_top','romance_flop']

df = pd.DataFrame()

for folder in folders:
    if folder[-3:]=='top':
        bs = 1
    else:
        bs = 0
    if folder[:3]=='sci':
        sf = 1
    else:
        sf = 0
        
    for text_file in os.listdir(path+folder+'/'):
        full_path = path + folder + '/' + text_file
        if text_file.endswith((".txt")):
            text  = open(full_path, 'r').read()
            temp = pd.DataFrame({
                    'best_seller': bs,
                    'sci_fi': sf,
                    'title': text_file[:-4].replace('_',' ').replace('-',' - '),
                    'body': text.decode('ascii', 'ignore').replace('\n',' ').replace('\r','')}, 
                                index=[0])
            df = pd.concat([df, temp])

In [126]:
df = df.reset_index()
del df['index']
df.head(2)

,best_seller,body,sci_fi,title
0,1,How to explain? How to describe? Even the omni...,1,a fire upon the deep - vernor vinge
1,1,"The body lay naked and facedown, a deathly gr...",1,ancillary justice - ann leckie


In [128]:
# check for front and back matter in body
# i have cleaned up the files by using this alert
# remaining issues are intentional usage of terms in the text

check_words = ['acknowledgements','table of contents','about the author', 'appendix', 
               'copyright','isbn','by this author']

for i in range(0,len(df)):
    for word in check_words:
        if word in df.iloc[i,1].lower():
            print df.ix[i,3], ' : ', word
        

a fire upon the deep - vernor vinge  :  appendix
dune - frank herbert  :  appendix
rainbows end - vernor vinge  :  table of contents
rainbows end - vernor vinge  :  appendix
rainbows end - vernor vinge  :  copyright
the algebraist - iain m banks  :  acknowledgements
the algebraist - iain m banks  :  appendix
atlanta nights - travis tea  :  appendix
fifty shades of grey - e l james  :  appendix
grey - e l james  :  appendix
outlander voyager - diana galbaldon  :  appendix


In [129]:
def avg_sentence_len(text):
    tokenizer = RegexpTokenizer(r'\w+')
    word_counts = []
    sentences = re.split(r'[.|!|?]', text)
    for sentence in sentences[:-1]:
        words = tokenizer.tokenize(sentence)
        word_counts.append(len(words))
    avg_word_count = sum(word_counts)/len(word_counts)  
    return avg_word_count

def word_count(text):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    return len(words)

def avg_word_len(text):
    letter_counts = []
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    for word in words:
        letter_counts.append(len(word))
    avg_word_len = sum(letter_counts)/len(letter_counts)
    return avg_word_len

def lex_div(text):
    tokenizer = RegexpTokenizer(r'\w+')
    words = tokenizer.tokenize(text)
    lexical_diversity = 1.0 * len(set(words)) / len(words)
    return lexical_diversity



In [130]:
df['avg_sent_len'] = df['body'].apply(avg_sentence_len)
df['word_count'] = df['body'].apply(word_count)
df['avg_word_len'] = df['body'].apply(avg_word_len)
df['lex_diversity'] = df['body'].apply(lex_div)

In [131]:
df.head()

,best_seller,body,sci_fi,title,avg_sent_len,word_count,avg_word_len,lex_diversity
0,1,How to explain? How to describe? Even the omni...,1,a fire upon the deep - vernor vinge,11,207548,4,0.064477
1,1,"The body lay naked and facedown, a deathly gr...",1,ancillary justice - ann leckie,11,108263,4,0.079815
2,1,Children of Dune Frank Herbert Muad'D...,1,children of dune - frank herbert,10,151352,4,0.076094
3,1,Consider Phlebas Iain Banks Idolatry is wor...,1,consider phlebas - iain m banks,14,176337,4,0.064042
4,1,"By nature, men are nearly alike; by practic...",1,diamond age - neal stephenson,17,190498,4,0.085061


In [132]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
best_seller,35.0,0.971429,0.169031,0.000000,1.000000,1.000000,1.000000,1.00000
sci_fi,35.0,0.571429,0.502096,0.000000,0.000000,1.000000,1.000000,1.00000
avg_sent_len,35.0,11.514286,2.800960,7.000000,10.000000,11.000000,13.000000,20.00000
word_count,35.0,136649.114286,73070.362563,25869.000000,91640.000000,120326.000000,174122.000000,394034.00000
avg_word_len,35.0,3.971429,0.169031,3.000000,4.000000,4.000000,4.000000,4.00000
lex_diversity,35.0,0.086743,0.023328,0.056274,0.071746,0.081857,0.098377,0.16398


In [133]:
sfbs = df[((df['best_seller']==1) & (df['sci_fi']==1))]
rmbs = df[((df['best_seller']==1) & (df['sci_fi']==0))]

In [134]:
sfbs.describe().T

,count,mean,std,min,25%,50%,75%,max
best_seller,19.0,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,1.000000
sci_fi,19.0,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,1.000000
avg_sent_len,19.0,11.421053,2.316985,9.000000,10.00000,10.000000,13.000000,17.000000
word_count,19.0,141674.000000,47392.036349,61927.000000,105744.00000,147721.000000,182357.500000,211455.000000
avg_word_len,19.0,4.000000,0.000000,4.000000,4.00000,4.000000,4.000000,4.000000
lex_diversity,19.0,0.086911,0.018780,0.064042,0.07218,0.081857,0.098377,0.121175


In [135]:
rmbs.describe().T

,count,mean,std,min,25%,50%,75%,max
best_seller,15.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.00000
sci_fi,15.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
avg_sent_len,15.0,11.600000,3.480558,7.000000,9.000000,11.000000,13.000000,20.00000
word_count,15.0,134483.200000,98774.725530,25869.000000,91640.000000,100512.000000,152186.000000,394034.00000
avg_word_len,15.0,3.933333,0.258199,3.000000,4.000000,4.000000,4.000000,4.00000
lex_diversity,15.0,0.083731,0.027064,0.056274,0.068227,0.074736,0.088196,0.16398


In [136]:
# initial feature review to see anything interesting

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(['said','didnt','know','don','dont','didn','did','ve','like',
                                           'enterprises','grey','ceo','232','le','christian'])

vect = TfidfVectorizer(ngram_range=(2,3), stop_words=stop_words)

summaries_sfbs = "".join(sfbs['body'])
summaries_rmbs = "".join(rmbs['body'])

ngrams_summaries_sfbs = vect.build_analyzer()(summaries_sfbs)
ngrams_summaries_rmbs = vect.build_analyzer()(summaries_rmbs)


In [137]:
# sci-fi books over-use character names
# also a long time ago, far far away
Counter(ngrams_summaries_sfbs).most_common(20)

[(u'hock seng', 579),
 (u'shook head', 470),
 (u'lieutenant awn', 397),
 (u'old man', 302),
 (u'long time', 266),
 (u'bene gesserit', 265),
 (u'princess nell', 242),
 (u'white shirts', 241),
 (u'years ago', 232),
 (u'judge fang', 210),
 (u'anaander mianaai', 201),
 (u'hiro says', 187),
 (u'muad dib', 181),
 (u'feyd rautha', 179),
 (u'thousand years', 171),
 (u'far away', 160),
 (u'reverend mother', 147),
 (u'pham nuwen', 147),
 (u'hong kong', 140),
 (u'uncle enzo', 132)]

In [138]:
# romance titles have some classics --> deep breath, closed eyes, emotional gestures
Counter(ngrams_summaries_rmbs).most_common(30)

[(u'shook head', 543),
 (u'miss steele', 468),
 (u'im going', 367),
 (u'anastasia steele', 323),
 (u'mr rochester', 286),
 (u'deep breath', 255),
 (u'im sorry', 251),
 (u'mrs jennings', 230),
 (u'young ian', 220),
 (u'im sure', 213),
 (u'tara jean', 206),
 (u'mr willoughby', 200),
 (u'long time', 198),
 (u'closed eyes', 197),
 (u'blue eyes', 173),
 (u'youre going', 164),
 (u'took deep', 158),
 (u'took deep breath', 140),
 (u'shaking head', 133),
 (u'yes sir', 132),
 (u'open door', 132),
 (u'colonel brandon', 130),
 (u'great deal', 130),
 (u'living room', 129),
 (u'steele subject', 127),
 (u'st john', 125),
 (u'young man', 123),
 (u'mrs dashwood', 121),
 (u'anastasia steele subject', 121),
 (u'turned away', 113)]

In [148]:

char_name_stop = stop_words.union(['awn','muad','dib','hock','seng','nell','pham','nuwen', 'enzo','anaander','mianaai',
                                   'rautha','feyd','willoughby','brandon','john','dashwood','jessica','fang','hiro',
                                  'steele','anastasia','rochester','jennings','ian','middleton','tara','jean','fairfax',
                                  'mcgraw','finkle'])

vect = TfidfVectorizer(ngram_range=(2,3), stop_words=char_name_stop)

summaries_sfbs = "".join(sfbs['body'])
summaries_rmbs = "".join(rmbs['body'])

ngrams_summaries_sfbs = vect.build_analyzer()(summaries_sfbs)
ngrams_summaries_rmbs = vect.build_analyzer()(summaries_rmbs)


In [149]:
Counter(ngrams_summaries_sfbs).most_common(30)

[(u'shook head', 470),
 (u'old man', 302),
 (u'long time', 266),
 (u'bene gesserit', 265),
 (u'white shirts', 241),
 (u'years ago', 232),
 (u'thousand years', 171),
 (u'far away', 160),
 (u'reverend mother', 147),
 (u'hong kong', 140),
 (u'shakes head', 124),
 (u'make sure', 124),
 (u'sjandra kei', 121),
 (u'lord radch', 120),
 (u'closed eyes', 119),
 (u'long ago', 119),
 (u'right hand', 116),
 (u'half hour', 115),
 (u'years old', 114),
 (u'im sure', 106),
 (u'unaha closp', 105),
 (u'time time', 103),
 (u'deep breath', 102),
 (u'light years', 100),
 (u'young man', 99),
 (u'half dozen', 99),
 (u'turned away', 98),
 (u'fwi song', 97),
 (u'quercer janath', 96),
 (u'inspector supervisor', 96)]

In [150]:
# this is funnier
Counter(ngrams_summaries_rmbs).most_common(30)

[(u'shook head', 543),
 (u'im going', 367),
 (u'deep breath', 255),
 (u'im sorry', 251),
 (u'im sure', 213),
 (u'long time', 198),
 (u'closed eyes', 197),
 (u'blue eyes', 173),
 (u'youre going', 164),
 (u'took deep', 158),
 (u'took deep breath', 140),
 (u'shaking head', 133),
 (u'yes sir', 132),
 (u'open door', 132),
 (u'great deal', 130),
 (u'living room', 129),
 (u'young man', 123),
 (u'turned away', 113),
 (u'oh aye', 112),
 (u'oh yes', 108),
 (u'eyes closed', 105),
 (u'eyes fixed', 101),
 (u'ive seen', 100),
 (u'im glad', 96),
 (u'ye ken', 94),
 (u'new york', 92),
 (u'little bit', 92),
 (u'years ago', 88),
 (u'gray eyes', 88),
 (u'cleared throat', 87)]